## Imports

In [1]:
import time
import names
import random
import pathlib
import numpy as np
import pandas as pd
import pandasql as pds

from datetime import datetime
from faker import Faker
from scraping import whisky_web_scraping


faker = Faker()

## Scraping data and writing into file

In [4]:
whisky_obj = whisky_web_scraping()
product_df_from_site = whisky_obj.scrape_whisky(number_of_pages=5)

In [ ]:
product_df_from_site_copy = product_df_from_site.copy()

In [ ]:
# creating directory for output
CURRENT_DIR = pathlib.Path().resolve()
OUTPUT_DIR = CURRENT_DIR / "whiskey_data"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

In [6]:
product_df_from_site_copy.to_csv('D:\Projects\Retail Sales DWH\etl\whiskey_data\whiskey_data.csv')

In [ ]:
product_df_from_site_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4382 entries, 0 to 4381
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Product_Name     4382 non-null   object
 1   Alcohol_Percent  4382 non-null   object
 2   Alcohol_Amount   4382 non-null   object
 3   Alcohol_Price    4382 non-null   object
dtypes: object(4)
memory usage: 137.1+ KB


In [ ]:
product_df_from_site_copy.head()

,Product_Name,Alcohol_Percent,Alcohol_Amount,Alcohol_Price
0,Glen Scotia Victoriana,54.2,70,72.95
1,Glen Scotia Double Cask,46,70,48.95
2,Glen Scotia Double Cask,46,70,39.95
3,Glen Scotia 12 Year Old Seasonal Release,53.3,70,79.95
4,Glen Scotia 15 Year Old,46,70,64.95


## Generating random data

### Generating product data

In [73]:
product_df_from_file = pd.read_csv('D:\Projects\Retail Sales DWH\etl\whiskey_data\whiskey_data.csv', index_col='Unnamed: 0')

In [74]:
product_df = product_df_from_file.copy() 

In [75]:
product_df.head()

,Product_Name,Alcohol_Percent,Alcohol_Amount,Alcohol_Price
0,Glen Scotia Victoriana,54.2,70.0,72.95
1,Glen Scotia Double Cask,46.0,70.0,48.95
2,Glen Scotia Double Cask,46.0,70.0,39.95
3,Glen Scotia 12 Year Old Seasonal Release,53.3,70.0,79.95
4,Glen Scotia 15 Year Old,46.0,70.0,64.95


In [76]:
# renaming columns
product_df.columns = [c.lower() for c in product_df.columns]
product_df.head()

,product_name,alcohol_percent,alcohol_amount,alcohol_price
0,Glen Scotia Victoriana,54.2,70.0,72.95
1,Glen Scotia Double Cask,46.0,70.0,48.95
2,Glen Scotia Double Cask,46.0,70.0,39.95
3,Glen Scotia 12 Year Old Seasonal Release,53.3,70.0,79.95
4,Glen Scotia 15 Year Old,46.0,70.0,64.95


In [77]:
# changing data type of alchocol_price collumn
product_df['alcohol_price'] = product_df['alcohol_price'].str.replace(',','').astype('float')

In [78]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4382 entries, 0 to 4381
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   product_name     4382 non-null   object 
 1   alcohol_percent  4382 non-null   float64
 2   alcohol_amount   4382 non-null   float64
 3   alcohol_price    4382 non-null   float64
dtypes: float64(3), object(1)
memory usage: 171.2+ KB


In [79]:
np.random.default_rng().choice(10, 10, replace=False)

array([4, 5, 2, 9, 3, 0, 8, 1, 6, 7], dtype=int64)

In [80]:
# Generate a column of unique product ids
product_id = np.random.default_rng().choice(len(product_df.product_name), len(product_df.product_name), replace = False)

# Verify that there are as many ids as there are products
assert len(set(product_id)) == len(product_df.product_name)

# Verify that the new ids are unique
assert len(pd.Series(product_id).unique()) == len(product_id)

# Insert the new column into the dataframe
product_df['product_id'] = product_id

In [81]:
product_df.head()

,product_name,alcohol_percent,alcohol_amount,alcohol_price,product_id
0,Glen Scotia Victoriana,54.2,70.0,72.95,565
1,Glen Scotia Double Cask,46.0,70.0,48.95,2398
2,Glen Scotia Double Cask,46.0,70.0,39.95,2919
3,Glen Scotia 12 Year Old Seasonal Release,53.3,70.0,79.95,3606
4,Glen Scotia 15 Year Old,46.0,70.0,64.95,1371


### Generating Employee Data

In [82]:
# Generating 100 Employee Unique id's
employee_id = np.random.default_rng().choice(4000, 100, replace = False)

# Verify that there are as many ids as there are employees
assert len(set(employee_id)) == 100

# Verify that the new ids are unique
assert len(pd.Series(employee_id).unique()) == len(employee_id)

In [83]:
# Generating 100 Employee Data
employee_first_name = []
employee_last_name = []
employee_full_name = []
employee_email = []
employee_city = []
departments = ['Sales', 'Finance', 'Marketing', 'BI']
employee_department = []

# iterate through the employees and generate random data
for i in range(len(employee_id)):
    employee_first_name.append(names.get_first_name())
    employee_last_name.append(names.get_last_name())
    employee_full_name.append(employee_first_name[i] + ' ' + employee_last_name[i])
    employee_email.append(employee_first_name[i] + employee_last_name[i][0].lower() + random.choice(['@gmail.com','@mail.ru','@yahoo.com']))
    employee_city.append(faker.city())
    employee_department.append(np.random.choice(departments, 1)[0])

In [84]:
# Create an employee dataframe
employee_df = pd.DataFrame(employee_id, columns = ['employee_id'])
employee_df['first_name'] = employee_first_name
employee_df['last_name'] = employee_last_name
employee_df['full_name'] = employee_full_name
employee_df['email'] = employee_email
employee_df['city'] = employee_city
employee_df['department'] = employee_department

In [85]:
employee_df.head()
employee_df.to_csv('files_for_medium/emp_df.csv')

### Generating Customer Data

In [86]:
# Generating 1000 Customer Unique id's
customer_id = np.random.default_rng().choice(999999, 1000, replace = False)

# Verify that there are as many ids as there are customers
assert len(set(customer_id)) == 1000

# Verify that the new ids are unique
assert len(pd.Series(customer_id).unique()) == len(customer_id)

In [87]:
# Generating 1000 Customers Data
customer_first_name = []
customer_last_name = []
customer_full_name = []
customer_email = []
customer_last_four_digits = []
customer_country = []
customer_country_code = []
customer_street = []
customer_credit_card_company = []


# iterate through the customers and generate random data
for i in range(len(customer_id)): 
    customer_first_name.append(names.get_first_name())
    customer_last_name.append(names.get_last_name())
    customer_full_name.append(customer_first_name[i] + ' ' + customer_last_name[i])
    customer_email.append(customer_first_name[i] + customer_last_name[i][0].lower() + random.choice(['@gmail.com','@mail.ru','@yahoo.com']))
    customer_last_four_digits.append(np.random.randint(low = 1000, high = 9999, size = 1)[0])
    customer_country.append(faker.country())
    customer_country_code.append(customer_country[i][0:3].upper())
    customer_street.append(faker.street_address())
    customer_credit_card_company.append(faker.credit_card_provider())

In [88]:
# Create a customer dataframe
customer_df = pd.DataFrame(customer_id, columns = ['customer_id'])
customer_df['first_name'] = customer_first_name
customer_df['last_name'] = customer_last_name
customer_df['full_name'] = customer_full_name
customer_df['email'] = customer_email
customer_df['country'] = customer_country
customer_df['country_code'] = customer_country_code
customer_df['street'] = customer_street
customer_df['credit_provider'] = customer_credit_card_company
customer_df['four_digits'] = customer_last_four_digits

In [89]:
customer_df.head()

,customer_id,first_name,last_name,full_name,email,country,country_code,street,credit_provider,four_digits
0,502010,Janice,Hare,Janice Hare,Janiceh@gmail.com,Botswana,BOT,9846 Herrera Spur,VISA 16 digit,6764
1,78215,Carmen,Winchell,Carmen Winchell,Carmenw@mail.ru,Cayman Islands,CAY,38686 Edward Manors Apt. 601,JCB 16 digit,2829
2,890124,Ann,Martin,Ann Martin,Annm@mail.ru,Brazil,BRA,5920 Nguyen Valley,Maestro,2751
3,194751,Jackie,Gonzalez,Jackie Gonzalez,Jackieg@mail.ru,Denmark,DEN,616 Robert Run Suite 717,Maestro,3550
4,266016,Vincent,Archibald,Vincent Archibald,Vincenta@gmail.com,British Virgin Islands,BRI,8625 Lindsay Bypass Apt. 840,Maestro,7521


### Generating Payment Data

In [90]:
# Generating random days in the range of 1990 to 2020
date_range = pd.date_range(start = "1990-01-01", end = "2022-12-31", freq="D")

In [91]:
len(date_range)

12053

In [92]:
# Generating Unique payment id's
payment_id = np.random.default_rng().choice(1_000_000, len(date_range), replace = False)

# Verify that there are as many ids as there are dates
assert len(set(payment_id)) == len(date_range)

# Verify that the new ids are unique
assert len(pd.Series(payment_id).unique()) == len(payment_id)

In [93]:
len(payment_id)

12053

In [94]:
# Generating payments Data
customer_id_payments = []
employee_id_payments = []
product_id_payments = []
dates = []


# iterate through the payments and generate random data
for i in range(len(payment_id)):
    dates.append(datetime.strftime(random.choice(date_range), format='%Y-%m-%d'))
    customer_id_payments.append(random.choice(customer_id))
    employee_id_payments.append(random.choice(employee_id))
    product_id_payments.append(random.choice(product_id))

In [95]:
# Create a payments dataframe
payment_df = pd.DataFrame(payment_id, columns = ['payment_id'])
payment_df['date'] = sorted(dates)
payment_df['customer_id'] = customer_id_payments
payment_df['employee_id'] = employee_id_payments
payment_df['product_id'] = product_id_payments

In [96]:
payment_df.head()

,payment_id,date,customer_id,employee_id,product_id
0,972299,1990-01-01,294813,929,2128
1,616074,1990-01-02,425087,790,905
2,240878,1990-01-04,569078,579,1671
3,364474,1990-01-05,946978,20,614
4,149395,1990-01-05,137018,3160,3553


In [97]:
def sql(query):
    return pds.sqldf(query)

In [98]:
# Adding the alcohol_price column to the table
query = '''
select p1.*, 
p2.alcohol_price as price
from payment_df p1
inner join product_df p2
on p1.product_id = p2.product_id
'''

payment_df = sql(query)

In [99]:
payment_df.head()

,payment_id,date,customer_id,employee_id,product_id,price
0,972299,1990-01-01,294813,929,2128,235.00
1,616074,1990-01-02,425087,790,905,4.45
2,240878,1990-01-04,569078,579,1671,21.95
3,364474,1990-01-05,946978,20,614,16.95
4,149395,1990-01-05,137018,3160,3553,52.95


## Designing the Central RDBMS and Normalizing the Data

In [100]:
product_df.head()

,product_name,alcohol_percent,alcohol_amount,alcohol_price,product_id
0,Glen Scotia Victoriana,54.2,70.0,72.95,565
1,Glen Scotia Double Cask,46.0,70.0,48.95,2398
2,Glen Scotia Double Cask,46.0,70.0,39.95,2919
3,Glen Scotia 12 Year Old Seasonal Release,53.3,70.0,79.95,3606
4,Glen Scotia 15 Year Old,46.0,70.0,64.95,1371


In [101]:
employee_df.head()

,employee_id,first_name,last_name,full_name,email,city,department
0,2657,Cathy,Mullen,Cathy Mullen,Cathym@mail.ru,New Alexandra,Marketing
1,2509,Jeannine,Brumback,Jeannine Brumback,Jeannineb@yahoo.com,Derekshire,Finance
2,1791,Catherine,Brazill,Catherine Brazill,Catherineb@yahoo.com,East Timothy,Finance
3,3160,Miriam,Sheehan,Miriam Sheehan,Miriams@gmail.com,West Nicholas,Marketing
4,2067,Sarah,Beatty,Sarah Beatty,Sarahb@gmail.com,Royfurt,BI


In [102]:
customer_df.head()

,customer_id,first_name,last_name,full_name,email,country,country_code,street,credit_provider,four_digits
0,502010,Janice,Hare,Janice Hare,Janiceh@gmail.com,Botswana,BOT,9846 Herrera Spur,VISA 16 digit,6764
1,78215,Carmen,Winchell,Carmen Winchell,Carmenw@mail.ru,Cayman Islands,CAY,38686 Edward Manors Apt. 601,JCB 16 digit,2829
2,890124,Ann,Martin,Ann Martin,Annm@mail.ru,Brazil,BRA,5920 Nguyen Valley,Maestro,2751
3,194751,Jackie,Gonzalez,Jackie Gonzalez,Jackieg@mail.ru,Denmark,DEN,616 Robert Run Suite 717,Maestro,3550
4,266016,Vincent,Archibald,Vincent Archibald,Vincenta@gmail.com,British Virgin Islands,BRI,8625 Lindsay Bypass Apt. 840,Maestro,7521


In [103]:
payment_df

,payment_id,date,customer_id,employee_id,product_id,price
0,972299,1990-01-01,294813,929,2128,235.00
1,616074,1990-01-02,425087,790,905,4.45
2,240878,1990-01-04,569078,579,1671,21.95
3,364474,1990-01-05,946978,20,614,16.95
4,149395,1990-01-05,137018,3160,3553,52.95
...,...,...,...,...,...,...
12048,264269,2022-12-28,94124,2416,4332,6.45
12049,909468,2022-12-29,444931,2657,626,41.75
12050,528749,2022-12-29,962382,3520,2919,39.95
12051,890769,2022-12-31,611988,1225,1006,42.75


### Normalizing customer_df table

#### First Table — countries

In [104]:
# Creating a new table called countries
unique_countries = customer_df.country.unique()
countries_df = pd.DataFrame(unique_countries, columns = ['Country'])
countries_df['Country_Code'] = countries_df.Country.str[0:3]
countries_df['Country_Code'] = countries_df.Country_Code.str.upper()
countries_df['country_id'] = [*range(0,len(countries_df))]
countries_df.tail()

,Country,Country_Code,country_id
231,Russian Federation,RUS,231
232,France,FRA,232
233,Mozambique,MOZ,233
234,India,IND,234
235,Guam,GUA,235


In [105]:
countries_df.to_csv('files_for_medium/countries_df.csv')

In [106]:
# Extracting the country_id column from customers
query = '''
select countries_df.country_id
from customer_df 
join countries_df
on 
    customer_df.country_code = countries_df.country_code and
    customer_df.country = countries_df.country
'''

country_ids = sql(query)


# Connecting countries to customers by adding the foregin key: country_id
customer_df['country_id'] = country_ids

In [107]:
customer_df = customer_df.drop(['country','country_code'],axis=1)
customer_df.head()

,customer_id,first_name,last_name,full_name,email,street,credit_provider,four_digits,country_id
0,502010,Janice,Hare,Janice Hare,Janiceh@gmail.com,9846 Herrera Spur,VISA 16 digit,6764,0
1,78215,Carmen,Winchell,Carmen Winchell,Carmenw@mail.ru,38686 Edward Manors Apt. 601,JCB 16 digit,2829,1
2,890124,Ann,Martin,Ann Martin,Annm@mail.ru,5920 Nguyen Valley,Maestro,2751,2
3,194751,Jackie,Gonzalez,Jackie Gonzalez,Jackieg@mail.ru,616 Robert Run Suite 717,Maestro,3550,3
4,266016,Vincent,Archibald,Vincent Archibald,Vincenta@gmail.com,8625 Lindsay Bypass Apt. 840,Maestro,7521,4


In [108]:
customer_df.head()

,customer_id,first_name,last_name,full_name,email,street,credit_provider,four_digits,country_id
0,502010,Janice,Hare,Janice Hare,Janiceh@gmail.com,9846 Herrera Spur,VISA 16 digit,6764,0
1,78215,Carmen,Winchell,Carmen Winchell,Carmenw@mail.ru,38686 Edward Manors Apt. 601,JCB 16 digit,2829,1
2,890124,Ann,Martin,Ann Martin,Annm@mail.ru,5920 Nguyen Valley,Maestro,2751,2
3,194751,Jackie,Gonzalez,Jackie Gonzalez,Jackieg@mail.ru,616 Robert Run Suite 717,Maestro,3550,3
4,266016,Vincent,Archibald,Vincent Archibald,Vincenta@gmail.com,8625 Lindsay Bypass Apt. 840,Maestro,7521,4


In [109]:
customer_df.to_csv('files_for_medium/customers_country_drop_df.csv')

#### Second table - customer_cc

In [110]:

# Creating a new table called customer_cc
unique_cc_providers = customer_df.credit_provider.unique()
customer_cc_df = pd.DataFrame(unique_cc_providers, columns = ['credit_provider'])
customer_cc_df['credit_provider_id'] = [*range(0,len(customer_cc_df))]
customer_cc_df.head()

,credit_provider,credit_provider_id
0,VISA 16 digit,0
1,JCB 16 digit,1
2,Maestro,2
3,American Express,3
4,JCB 15 digit,4


In [111]:
customer_cc_df.to_csv('files_for_medium/cc_df.csv')

In [112]:
# Extracting the credit_provider_id column from customers
query = '''
select customer_cc_df.credit_provider_id
from customer_df 
join customer_cc_df
on 
    customer_df.credit_provider = customer_cc_df.credit_provider
'''

credit_provider_id = sql(query)

# Connecting customer_cc to customers by adding the foregin key: credit_provider_id 
customer_df['credit_provider_id'] = credit_provider_id

# Dropping the column credit_provider
customer_df = customer_df.drop(['credit_provider'],axis=1)

In [113]:
customer_df.head()

,customer_id,first_name,last_name,full_name,email,street,four_digits,country_id,credit_provider_id
0,502010,Janice,Hare,Janice Hare,Janiceh@gmail.com,9846 Herrera Spur,6764,0,0
1,78215,Carmen,Winchell,Carmen Winchell,Carmenw@mail.ru,38686 Edward Manors Apt. 601,2829,1,1
2,890124,Ann,Martin,Ann Martin,Annm@mail.ru,5920 Nguyen Valley,2751,2,2
3,194751,Jackie,Gonzalez,Jackie Gonzalez,Jackieg@mail.ru,616 Robert Run Suite 717,3550,3,2
4,266016,Vincent,Archibald,Vincent Archibald,Vincenta@gmail.com,8625 Lindsay Bypass Apt. 840,7521,4,2


### Third Table - departaments

In [114]:
employee_df.head()

,employee_id,first_name,last_name,full_name,email,city,department
0,2657,Cathy,Mullen,Cathy Mullen,Cathym@mail.ru,New Alexandra,Marketing
1,2509,Jeannine,Brumback,Jeannine Brumback,Jeannineb@yahoo.com,Derekshire,Finance
2,1791,Catherine,Brazill,Catherine Brazill,Catherineb@yahoo.com,East Timothy,Finance
3,3160,Miriam,Sheehan,Miriam Sheehan,Miriams@gmail.com,West Nicholas,Marketing
4,2067,Sarah,Beatty,Sarah Beatty,Sarahb@gmail.com,Royfurt,BI


In [115]:
# Extracting the departments from the employees table
departments = pd.Series(employee_df.department.unique()).to_list()

# Generating unique department ids
department_id = [*range(0, len(departments))]

# Creating a table called departments
department_df = pd.DataFrame(department_id, columns=['department_id'])
department_df['department'] = departments

In [116]:
# Extracting the country_id column from customers
query = '''
select department_df.department_id
from employee_df 
join department_df
on 
    employee_df.department = department_df.department
'''

department_ids = sql(query)

# Connecting countries to customers by adding the foregin key: country_id
employee_df['department_id'] = department_ids

In [117]:
# Dropping the column department
employee_df = employee_df.drop('department',axis = 1)

In [119]:
employee_df.head().to_csv('files_for_medium/final_emps.csv')

## Loading the data into the central RDBMS

### Connecting Python to MySQL

In [122]:
# Connecting Python to MySQL
import pymysql
import pandas as pd

import os
from dotenv import load_dotenv

load_dotenv()

mysql_pass = os.getenv('mysql_pass')

connection = pymysql.connect(host ='localhost',port=int(3306),user='root',passwd=str(mysql_pass))

# # # Creating a cursor object
cursor = connection.cursor()

### Creating a new Schema

In [123]:
# Create a new schema called whiskey_shop
cursor.execute('''
drop schema if exists whiskey_retail_shop;
''')

cursor.execute('''
create schema whiskey_retail_shop;
''')

# Use the new schema
cursor.execute('''
use whiskey_retail_shop;
''')

0

### Creating empty tables

In [124]:
cursor.execute('''
DROP TABLE IF EXISTS countries;
''')

cursor.execute('''
CREATE TABLE countries (
    Country VARCHAR(100) NOT NULL,
    Country_Code VARCHAR(100) NOT NULL,
    country_id INT PRIMARY KEY
    );
''')

0

In [125]:
cursor.execute('''
DROP TABLE IF EXISTS customer_cc;
''')

cursor.execute('''
CREATE TABLE customer_cc (
    credit_provider VARCHAR(100) NOT NULL,
    credit_provider_id INT PRIMARY KEY
    );
''')

0

In [126]:
cursor.execute('''
DROP TABLE IF EXISTS products;
''')

cursor.execute('''
CREATE TABLE products (
    Product_Name VARCHAR(100) NOT NULL,
    Alcohol_Percent FLOAT NOT NULL,
    Alcohol_Amount FLOAT NOT NULL,
    Alcohol_Price FLOAT NOT NULL,
    product_id int NOT NULL PRIMARY KEY
    );
''')

0

In [127]:
cursor.execute('''
DROP TABLE IF EXISTS departments;
''')

cursor.execute('''
CREATE TABLE departments (
    department_id INT PRIMARY KEY,
    department VARCHAR(100) NOT NULL
    );
''')

0

In [128]:
cursor.execute('''
DROP TABLE IF EXISTS customers;
''')

cursor.execute('''
CREATE TABLE customers (
    customer_id INT PRIMARY KEY NOT NULL,
    first_name VARCHAR(100) NOT NULL,
    last_name VARCHAR(100) NOT NULL,
    full_name VARCHAR(100) NOT NULL,
    email VARCHAR(100) NOT NULL,
    street VARCHAR(100) NOT NULL,
    four_digits INT NOT NULL,
    country_id INT NOT NULL,
    credit_provider_id INT NOT NULL,
    
    FOREIGN KEY (country_id) REFERENCES countries (country_id),
    FOREIGN KEY (credit_provider_id) REFERENCES customer_cc (credit_provider_id)
);
''')

0

In [129]:
cursor.execute('''
DROP TABLE IF EXISTS employees;
''')

cursor.execute('''
CREATE TABLE employees (
    employee_id INT PRIMARY KEY NOT NULL,
    first_name VARCHAR(100) NOT NULL,
    last_name VARCHAR(100) NOT NULL,
    full_name VARCHAR(100) NOT NULL,
    email VARCHAR(100) NOT NULL,
    city VARCHAR(100) NOT NULL,
    department_id INT NOT NULL,
    
    FOREIGN KEY (department_id) REFERENCES departments(department_id)
);
''')

0

In [130]:
cursor.execute('''
DROP TABLE IF EXISTS payments;
''')

cursor.execute('''
CREATE TABLE payments (
    payment_id INT NOT NULL PRIMARY KEY,
    date DATE NOT NULL,
    customer_id INT NOT NULL,
    employee_id INT NOT NULL,
    product_id INT NOT NULL,
    price FLOAT NOT NULL,

    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
    );
''')

0

### Populating the tables

In [131]:
# Convert the Dataframe into a list of arrays
records = countries_df.to_records(index=False)

# Convert the list of arrays into a tuple of tuples
result = tuple(records)

for data in range(0,len(result)):
    
    # Create a new record
    query = "insert into countries (country, country_code, country_id) values {}".format(result[data])
    
    # Execute the query
    cursor.execute(query)
    
    
# Commit the transaction
connection.commit()

In [132]:
# Convert the Dataframe into a list of arrays
records = customer_cc_df.to_records(index=False)

# Convert the list of arrays into a tuple of tuples
result = tuple(records)

for data in range(0,len(result)):
    
    # Create a new record
    query = "insert into customer_cc (credit_provider, credit_provider_id) values {}".format(result[data])
    
    # Execute the query
    cursor.execute(query)
    
    
# Commit the transaction
connection.commit()

In [133]:
# Convert the Dataframe into a list of arrays
records = product_df.to_records(index=False)

# Convert the list of arrays into a tuple of tuples
result = tuple(records)

for data in range(0,len(result)):
    
    # Create a new record
    query = "insert into products (Product_Name, Alcohol_Percent, Alcohol_Amount, Alcohol_Price,product_id) values {}".format(result[data])
    
    # Execute the query
    cursor.execute(query)
    
    
# Commit the transaction
connection.commit()

In [134]:
# Convert the Dataframe into a list of arrays
records = department_df.to_records(index=False)

# Convert the list of arrays into a tuple of tuples
result = tuple(records)

for data in range(0,len(result)):
    
    # Create a new record
    query = "insert into departments (department_id, department) values {}".format(result[data])
    
    # Execute the query
    cursor.execute(query)
    
    
# Commit the transaction
connection.commit()

In [135]:
# Convert the Dataframe into a list of arrays
records = customer_df.to_records(index=False)

# Convert the list of arrays into a tuple of tuples
result = tuple(records)

for data in range(0,len(result)):
    
    # Create a new record
    query = "insert into customers (customer_id, first_name, last_name, full_name, email, street, four_digits, country_id, credit_provider_id) values {}".format(result[data])
    
    # Execute the query
    cursor.execute(query)
    
    
# Commit the transaction
connection.commit()

In [136]:
# Convert the Dataframe into a list of arrays
records = employee_df.to_records(index=False)

# Convert the list of arrays into a tuple of tuples
result = tuple(records)

for data in range(0,len(result)):
    
    # Create a new record
    query = "insert into employees (employee_id, first_name, last_name, full_name,email,city, department_id) values {}".format(result[data])
    
    # Execute the query
    cursor.execute(query)
    
    
# Commit the transaction
connection.commit()

In [137]:
# Convert the Dataframe into a list of arrays
records = payment_df.to_records(index=False)

# Convert the list of arrays into a tuple of tuples
result = tuple(records)

for data in range(0,len(result)):
    
    # Create a new record
    query = "insert into payments (payment_id, date,customer_id,employee_id,product_id,price) values {}".format(result[data])
    
    # Execute the query
    cursor.execute(query)
    
    
# Commit the transaction
connection.commit()

In [138]:
customer_df.to_csv('files_for_medium\customer_df.csv')